In [ ]:
!poetry install -q

In [ ]:
import os

from configurations.chat_openai import *
from configurations.cypher_chat_openai import *
from configurations.default import *
from configurations.neo4j import *
from configurations.openai_embeddings import *

configurations = Configuration.load('configuration.yaml')

default = DefaultConfiguration.grab(configurations, '')
neo4j = Neo4jConfiguration.grab(configurations)
chat_openai = ChatOpenAIConfiguration.grab(configurations)
openai_embeddings = OpenAIEmbeddingsConfiguration.grab(configurations)
cypher_chat_openai = CypherChatOpenAIConfiguration.grab(configurations)

os.environ['OPENAI_API_KEY'] = default.openai_api_key

In [ ]:
from langchain.schema import HumanMessage, SystemMessage, AIMessage
from langchain_openai import *
from package.utilities import *

chat = ChatOpenAI(**chat_openai.dict())
entities = get_file_contents('entities.txt').splitlines()

In [ ]:
orchestrator_prompt = get_file_contents('orchestrator-prompt.txt')
aiMessage: AIMessage = chat.invoke([
    SystemMessage(content=orchestrator_prompt),
    HumanMessage(content="The manager of chatbot's project has changed, now it's Ivan Petrov")
])
candidates = [candidate.lstrip('0123456789.- ') for candidate in aiMessage.content.splitlines()]
print(aiMessage.content)
print(candidates)

In [ ]:
# from langchain_community.vectorstores import FAISS
# 
# embeddings = OpenAIEmbeddings(model='text-embedding-ada-002', headers={'x-api-key': OPENAI_API_KEY})
# retriever = FAISS.from_texts(entities, embeddings).as_retriever()
# for candidate in candidates:
#     print(candidate, ' -> ', retriever.invoke(candidate)[0].page_content)

In [ ]:
from langchain_openai import *
from langchain_community.vectorstores import Neo4jVector

name = 'name'
title = 'title'

embeddings2 = OpenAIEmbeddings(**openai_embeddings.dict())
vector_index = Neo4jVector.from_existing_graph(
    embeddings2,
    **neo4j.dict(),
    # index_name='node_text_index_title',
    node_label='Person',
    text_node_properties=[title],
    embedding_node_property='embedding',
)
for candidate in candidates:
    response = vector_index.similarity_search(
        candidate
    )
    # print(candidate, ' -> ', response[0].page_content.strip().removeprefix(f"{title}: "))
    print(candidate, ' -> ', response)

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_community.graphs import Neo4jGraph
from langchain_community.chains.graph_qa.cypher import GraphCypherQAChain

graph = Neo4jGraph(**neo4j.dict())
nodes = graph.query("MATCH (n:Project) RETURN n")
# for node in nodes:
#     print([value['title'] for value in node.values()][0])
project_list = [value['title'] for node in nodes for value in node.values()]

CYPHER_GENERATION_TEMPLATE = f"""Task:Generate Cypher statement to query a graph database.
Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.
For project use only similarity name from this list: {project_list}
Schema:
{{schema}}
Note: Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement.

The question is:
{{question}}"""

CYPHER_GENERATION_PROMPT = PromptTemplate(input_variables=['schema', 'question'], validate_template=True, template=CYPHER_GENERATION_TEMPLATE)

chain = GraphCypherQAChain.from_llm(
    ChatOpenAI(**cypher_chat_openai.dict()),
    cypher_prompt=CYPHER_GENERATION_PROMPT,
    graph=graph,
    verbose=True,
    allow_dangerous_requests=True,
)
# embeddings4o = ChatOpenAI(model=default.chat_openai_model, openai_api_key=OPENAI_API_KEY, temperature=0)
# vector_store = Neo4jVector(embeddings=embeddings4o, graph=graph)
# llm = ChatOpenAI(model='gpt-4o', temperature=0, openai_api_key=OPENAI_API_KEY)
# chain1 = GraphCypherQAChain.from_llm(vector_store=vector_index, llm=llm, verbose=True)

response = chain.invoke({"query": "The director of chatbot project has been changed, now it's Ivan Petrov"})
# response = chain.invoke({"query": "Appoint Ivan Petrov as chatbot project manager"})
# response = chain.invoke({"query": "Remove all relations"})
# response = chain.invoke({"query": "Set relation: term with name manager can works in term with name department"})
# response

In [ ]:
from langchain_community.vectorstores import FAISS

graph = Neo4jGraph(**neo4j.dict())

entities = [d['label'] for d in graph.query(
    """
    MATCH (n)
    UNWIND labels(n) AS label
    RETURN DISTINCT label
    """)]
print(entities)
embeddings = OpenAIEmbeddings(**openai_embeddings.dict())
retriever = FAISS.from_texts(entities, embeddings).as_retriever()
# for candidate in candidates:
#     print(candidate, ' -> ', retriever.invoke(candidate)[0].page_content)
labels = list(set([retriever.invoke(candidate)[0].page_content for candidate in candidates]))
print(labels)

name = 'name'
title = 'title'

embeddings2 = OpenAIEmbeddings(**openai_embeddings.dict())
for label in labels:
    print(candidates)
    vector_index = Neo4jVector.from_existing_graph(
        embeddings2,
        **neo4j.dict(),
        index_name='manager_index',
        node_label=label,
        text_node_properties=[name, title],
        embedding_node_property='embedding',
    )
    for candidate in candidates:
        response = vector_index.similarity_search(candidate, k=1)
        print(candidate, label, '->', response[0].page_content.strip())

In [ ]:
from uuid import uuid4
from peewee import *

db = SqliteDatabase('database.sqlite')

class BaseModel(Model):
    class Meta:
        database = db
        legacy_table_names = False

class Entity(BaseModel):
    id = UUIDField(primary_key=True, default=uuid4)
    type = CharField()  # entity title from Neo4j
    title = CharField()

class Relation(BaseModel):
    from_ = ForeignKeyField(Entity, backref='relations', db_column='from')
    to = ForeignKeyField(Entity, backref='relations', db_column='to')
    type = CharField()  # relation name from §

    class Meta:
        primary_key = CompositeKey('from_', 'to', 'type')

db.connect(True)
db.create_tables([Entity, Relation])
Entity.create_table()
# a = Entity.select().count()
# print(a)    

In [ ]:
# Entity.create(title='New one', type='3db3c5bc-5431-425c-ae2a-00e4603a4578')
# Entity.create(title='Old two', type='4db3c5bc-5431-425c-ae2a-00e4603a4578')
# Relation.create(from_='3db3c5bc-5431-425c-ae2a-00e4603a4578', to='4db3c5bc-5431-425c-ae2a-00e4603a4578', type='Friend')
# Relation.create(from_='4db3c5bc-5431-425c-ae2a-00e4603a4578', to='4db3c5bc-5431-425c-ae2a-00e4603a4578', type='Friend')

In [ ]:
from package.table_generator import *

attach_table_generator(db)

dataFoo = db.generate_table(Entity, 'foo2', {'new_field': CharField()})
# dataBar = db.generate_table(Relation, 'bar2')

In [ ]:
import os
os.getcwd()